In [4]:
import glob
import pandas as pd
import json
import os

find_path = "/Users/joon/Desktop/딥러닝/데이터/Training/라벨링데이터/abnormal_230303_add/TL7"
file_list = glob.glob(os.path.join(find_path, "*/*/label/*.json"))

len(file_list)

60443

In [13]:
# 모든 데이터에 있는 최대 occupant수 확인
max_occupant = 1

for file in file_list:
    with open(file) as f:
        js = json.loads(f.read())
    occupant_info = js['occupant_info']
    num_occupant = len(occupant_info)

    if num_occupant > max_occupant:
        max_occupant = num_occupant
            
print(max_occupant)

2


# 데이터 변환

In [90]:
datas = []

for file in file_list:
    with open(file) as f:
        js = json.loads(f.read())

    metadata = js['metadata']
    scene_info = js['scene_info']
    creator = metadata['creator']

    occupant_info = js['occupant_info']
    num_occupant = len(occupant_info)

    video_id = metadata['video_id']
    date = metadata['date']
    scene_id = scene_info['scene_id']
    category_id = scene_info['category_id']
    category_name = scene_info['category_name']

    occupants_info = []
    occupants_id = []
    for occupant in occupant_info:
        occupant_id = occupant['occupant_id']
        occupant_position = occupant['occupant_posotion'] # posotion이라고 잘못되어있음

        occupants_info.append({'occupant_id': occupant_id, 'occupant_position': occupant_position})
        occupants_id.append(occupant_id)
        
    scene = js['scene']['data']
    for img in scene:
        data_length = len(scene)
        img_id = img['img_name']
        
        occupants = img['occupant']
        actions = ['없음' for _ in range(max_occupant)]
        for occupant in occupants:

            # occupant_info에 있는 순서
            idx = occupants_id.index(occupant['occupant_id'])

            occupants_info[idx]['bbox'] = occupant['body_b_box']
            occupants_info[idx]['fbox'] = occupant['face_b_box']
            occupants_info[idx]['action'] = occupant['action']
            actions[idx] = occupant['action']

        data = {"creator": creator, "date": date, "category_name": category_name, "data_length": data_length, "video_id": video_id, "scene_id": scene_id, "img_id": img_id, "scene_occupants": num_occupant, "img_occupants": len(occupants), "occupants_info": occupants_info.copy()} # copy가 문제가 아닌가? 확인 필요

        for i in range(max_occupant):
            data[f"action_{i+1}"] = actions[i]

        datas.append(data)

df = pd.DataFrame(data=datas)
df



,creator,date,category_name,data_length,video_id,scene_id,img_id,scene_occupants,img_occupants,occupants_info,action_1,action_2
0,디투리소스,20220124,휴대폰조작,5,SGA2101512,SGA2101512S0301,SGA2101512S0301IMG0001.jpg,1,1,"[{'occupant_id': 'occupant1', 'occupant_positi...",힐끗거리다,없음
1,디투리소스,20220124,휴대폰조작,5,SGA2101512,SGA2101512S0301,SGA2101512S0301IMG0002.jpg,1,1,"[{'occupant_id': 'occupant1', 'occupant_positi...",손을뻗다,없음
2,디투리소스,20220124,휴대폰조작,5,SGA2101512,SGA2101512S0301,SGA2101512S0301IMG0003.jpg,1,1,"[{'occupant_id': 'occupant1', 'occupant_positi...",손을뻗다,없음
3,디투리소스,20220124,휴대폰조작,5,SGA2101512,SGA2101512S0301,SGA2101512S0301IMG0004.jpg,1,1,"[{'occupant_id': 'occupant1', 'occupant_positi...",손을뻗다,없음
4,디투리소스,20220124,휴대폰조작,5,SGA2101512,SGA2101512S0301,SGA2101512S0301IMG0005.jpg,1,1,"[{'occupant_id': 'occupant1', 'occupant_positi...",손을뻗다,없음
...,...,...,...,...,...,...,...,...,...,...,...,...
302210,디투리소스,20220124,통화,5,SGA2101505,SGA2101505S0201,SGA2101505S0201IMG0001.jpg,1,1,"[{'occupant_id': 'occupant1', 'occupant_positi...",무언가를쥐다,없음
302211,디투리소스,20220124,통화,5,SGA2101505,SGA2101505S0201,SGA2101505S0201IMG0002.jpg,1,1,"[{'occupant_id': 'occupant1', 'occupant_positi...",무언가를쥐다,없음
302212,디투리소스,20220124,통화,5,SGA2101505,SGA2101505S0201,SGA2101505S0201IMG0003.jpg,1,1,"[{'occupant_id': 'occupant1', 'occupant_positi...",무언가를쥐다,없음
302213,디투리소스,20220124,통화,5,SGA2101505,SGA2101505S0201,SGA2101505S0201IMG0004.jpg,1,1,"[{'occupant_id': 'occupant1', 'occupant_positi...",운전하다,없음


In [51]:
df.loc[0, 'occupants_info']

[{'occupant_id': 'occupant1',
  'occupant_position': 'front',
  'bbox': [672.6399902343751,
   262.39573333333294,
   441.60000000000014,
   450.88967233072964],
  'fbox': [769.6, 260.7928888888891, 160, 190.32576121961793],
  'action': '손을뻗다'}]

In [52]:
df.loc[11505, 'occupants_info']

[{'occupant_id': 'occupant1',
  'occupant_position': 'front',
  'bbox': [655.67998046875,
   113.61991111111107,
   624.3200195312501,
   606.380088888889],
  'fbox': [879.67998046875, 115.27253333333323, 254.4, 345.4375834418405],
  'action': '운전하다'},
 {'occupant_id': 'occupant2',
  'occupant_position': 'back',
  'bbox': [110.07998046874998,
   177.61991111111112,
   299.20000000000005,
   542.380088888889],
  'fbox': [183.67998046875002, 180.95900566406252, 166.4, 174.24497777777785],
  'action': '물건을쥐다/휘드르다'}]

In [47]:
df.to_csv('./train_label_7.csv', index=False)

In [91]:
i = 0
print(df.loc[i, 'occupants_info'])
print(df.loc[i, 'action_1'])

[{'occupant_id': 'occupant1', 'occupant_position': 'front', 'bbox': [672.6399902343751, 262.39573333333294, 441.60000000000014, 450.88967233072964], 'fbox': [769.6, 260.7928888888891, 160, 190.32576121961793], 'action': '손을뻗다'}]
힐끗거리다


In [88]:
df.loc[0, :]

creator                                                        디투리소스
date                                                        20220124
category_name                                                  휴대폰조작
data_length                                                        5
video_id                                                  SGA2101512
scene_id                                             SGA2101512S0301
img_id                                    SGA2101512S0301IMG0001.jpg
scene_occupants                                                    1
img_occupants                                                      1
occupants_info     [{'occupant_id': 'occupant1', 'occupant_positi...
action_1                                                       힐끗거리다
action_2                                                          없음
Name: 0, dtype: object